In [1]:
import sys
import os
import itertools
sys.path.append(os.path.abspath('../'))
from manticore_handler import manticore_handler
from IPython.utils import io
import time

In [3]:
m = manticore_handler('thermostatInvariant.sol')

# -- Deploying Contract thermostatInvariant.sol
(level _is_sat_z3_call) took 0.0005061626434326172 seconds (sat)
# -- Contract Deployed      (took 122.25977635383606 seconds)


In [4]:
metodos = ["StartThermostat","SetTargetTemperature","SetMode"]
precondiciones = [x+"_precondition" for x in metodos]
def call_pre():
    for cond in precondiciones:
        m.callContractFunction(cond,tx_sender=m.witness_account)

In [5]:
states = list(itertools.product([0,1],repeat=len(metodos)))
def repr_state(state):
    text = ""
    for x,method in zip(state,metodos):
        text += '_'+method if x else ""
    if text == "":
        text = "vacio"
    return text

In [6]:
call_pre()

(level _is_sat_z3_call) took 0.0012106895446777344 seconds (sat)
(level _getvalue_z3_call) took 0.0003075599670410156 seconds
(level _is_sat_z3_call) took 0.002614736557006836 seconds (unsat)
(level _is_sat_z3_call) took 0.0038232803344726562 seconds (sat)
(level _getvalue_z3_call) took 0.0006229877471923828 seconds
(level _is_sat_z3_call) took 0.00015854835510253906 seconds (unsat)
(level _is_sat_z3_call) took 0.004094362258911133 seconds (sat)
(level _getvalue_z3_call) took 0.0006630420684814453 seconds
(level _is_sat_z3_call) took 0.0001742839813232422 seconds (unsat)


In [7]:
found = []
with io.capture_output() as captured:    
    for state in states:
        count = m.generateTestCases(keys=precondiciones,
                                    targets=state,
                                    testcaseName=f"STATE_{repr_state(state)}")
        if count > 0:
            found.append(f"found {count} testcases that reach {repr_state(state)} initial state")
        else:
            found.append(f"found no testcases for {repr_state(state)} initial state")

In [8]:
for line in found:
    print(line)

found no testcases for vacio initial state
found no testcases for _SetMode initial state
found no testcases for _SetTargetTemperature initial state
found no testcases for _SetTargetTemperature_SetMode initial state
found True testcases that reach _StartThermostat initial state
found no testcases for _StartThermostat_SetMode initial state
found no testcases for _StartThermostat_SetTargetTemperature initial state
found no testcases for _StartThermostat_SetTargetTemperature_SetMode initial state


In [9]:
m.take_snapshot()
start_setter = time.time()
with io.capture_output() as captured:
    m.callContractFunction("setter")
print(time.time()-start_setter)

568.4337089061737


In [10]:
call_pre()

(level _is_sat_z3_call) took 0.06100749969482422 seconds (sat)
(level _is_sat_z3_call) took 0.1050114631652832 seconds (sat)
(level _is_sat_z3_call) took 0.08168864250183105 seconds (sat)
(level _is_sat_z3_call) took 0.05533099174499512 seconds (sat)
(level _is_sat_z3_call) took 0.06388425827026367 seconds (sat)
(level _is_sat_z3_call) took 0.1288609504699707 seconds (sat)
(level _getvalue_z3_call) took 0.027181148529052734 seconds
(level _is_sat_z3_call) took 0.4247097969055176 seconds (unsat)
(level _is_sat_z3_call) took 0.047796010971069336 seconds (sat)
(level _getvalue_z3_call) took 0.015410423278808594 seconds
(level _is_sat_z3_call) took 0.23485207557678223 seconds (unsat)
(level _is_sat_z3_call) took 0.09161663055419922 seconds (sat)
(level _getvalue_z3_call) took 0.005263566970825195 seconds
(level _is_sat_z3_call) took 0.11105608940124512 seconds (unsat)
(level _is_sat_z3_call) took 0.13655447959899902 seconds (sat)
(level _getvalue_z3_call) took 0.2601492404937744 seconds
(l

In [11]:
m.take_snapshot()
with io.capture_output() as captured:    
    m.callContractFunction("StartThermostat")
call_pre()

(level _is_sat_z3_call) took 0.13864493370056152 seconds (sat)
(level _is_sat_z3_call) took 0.25679922103881836 seconds (sat)
(level _is_sat_z3_call) took 0.20386910438537598 seconds (sat)
(level _is_sat_z3_call) took 0.14404773712158203 seconds (sat)
(level _getvalue_z3_call) took 0.18724656105041504 seconds
(level _is_sat_z3_call) took 0.18120431900024414 seconds (unsat)
(level _is_sat_z3_call) took 0.212446928024292 seconds (sat)
(level _getvalue_z3_call) took 0.5550563335418701 seconds
(level _is_sat_z3_call) took 0.14444637298583984 seconds (unsat)
(level _is_sat_z3_call) took 0.2552485466003418 seconds (sat)
(level _getvalue_z3_call) took 0.04297780990600586 seconds
(level _is_sat_z3_call) took 0.15959548950195312 seconds (unsat)
(level _is_sat_z3_call) took 0.18941116333007812 seconds (sat)
(level _getvalue_z3_call) took 0.0413365364074707 seconds
(level _is_sat_z3_call) took 0.14867424964904785 seconds (unsat)
(level _is_sat_z3_call) took 0.11264634132385254 seconds (sat)
(leve

In [12]:
def repr_transition(state1,state2,metodo):
    return f"{repr_state(state1)}->{metodo}->{repr_state(state2)}"

In [13]:
ini_state = (1,0,0) #_StartThermostat
found = []
with io.capture_output() as captured:    
    for fin_state in states:
        count = m.generateTestCases(keys=(precondiciones+precondiciones),
                                    targets=(ini_state+fin_state),
                                    testcaseName=f"TRANSITION_{repr_transition(ini_state,fin_state,'push')}")
        if count > 0:
            found.append(f"found {count} testcases for transition {repr_transition(ini_state,fin_state,'push')}")
        else:
            found.append(f"found no testcases for transition {repr_transition(ini_state,fin_state,'push')}")

In [14]:
for line in found:
    print(line)

found no testcases for transition _StartThermostat->push->vacio
found no testcases for transition _StartThermostat->push->_SetMode
found no testcases for transition _StartThermostat->push->_SetTargetTemperature
found True testcases for transition _StartThermostat->push->_SetTargetTemperature_SetMode
found no testcases for transition _StartThermostat->push->_StartThermostat
found no testcases for transition _StartThermostat->push->_StartThermostat_SetMode
found no testcases for transition _StartThermostat->push->_StartThermostat_SetTargetTemperature
found no testcases for transition _StartThermostat->push->_StartThermostat_SetTargetTemperature_SetMode


In [15]:
m.goto_snapshot()
m.take_snapshot()
m.callContractFunction("SetTargetTemperature")
call_pre()

(level _is_sat_z3_call) took 0.09340023994445801 seconds (sat)
(level _is_sat_z3_call) took 0.06555438041687012 seconds (sat)
(level _is_sat_z3_call) took 0.2247450351715088 seconds (sat)
(level _getvalue_z3_call) took 0.09061074256896973 seconds
(level _getvalue_z3_call) took 0.0452115535736084 seconds
(level _getvalue_z3_call) took 0.012120246887207031 seconds
(level _is_sat_z3_call) took 0.23920941352844238 seconds (sat)
(level _is_sat_z3_call) took 0.09890174865722656 seconds (sat)
(level _getvalue_z3_call) took 0.0009434223175048828 seconds
(level _is_sat_z3_call) took 0.32235169410705566 seconds (sat)
(level _is_sat_z3_call) took 0.1276390552520752 seconds (sat)
(level _getvalue_z3_call) took 0.001569986343383789 seconds
(level _is_sat_z3_call) took 0.13506436347961426 seconds (sat)
(level _getvalue_z3_call) took 0.09399056434631348 seconds
(level _getvalue_z3_call) took 0.008403778076171875 seconds
(level _is_sat_z3_call) took 0.1590883731842041 seconds (sat)
(level _is_sat_z3_c

(level _is_sat_z3_call) took 0.36652684211730957 seconds (unsat)
(level _is_sat_z3_call) took 0.2082371711730957 seconds (sat)
(level _getvalue_z3_call) took 0.04930567741394043 seconds
(level _is_sat_z3_call) took 0.2010188102722168 seconds (unsat)
(level _is_sat_z3_call) took 1.237879753112793 seconds (sat)
(level _getvalue_z3_call) took 0.25728917121887207 seconds
(level _is_sat_z3_call) took 0.24759197235107422 seconds (unsat)
(level _is_sat_z3_call) took 0.19670724868774414 seconds (sat)
(level _is_sat_z3_call) took 0.46720457077026367 seconds (sat)
(level _getvalue_z3_call) took 0.005773067474365234 seconds
(level _getvalue_z3_call) took 0.2879598140716553 seconds
(level _is_sat_z3_call) took 0.11132383346557617 seconds (unsat)
(level _is_sat_z3_call) took 0.23842430114746094 seconds (sat)
(level _getvalue_z3_call) took 0.053600311279296875 seconds
(level _is_sat_z3_call) took 0.0310819149017334 seconds (unsat)
(level _is_sat_z3_call) took 0.2754354476928711 seconds (sat)
(level 

(level _is_sat_z3_call) took 0.3309366703033447 seconds (sat)
(level _is_sat_z3_call) took 0.05315589904785156 seconds (sat)
(level _getvalue_z3_call) took 0.3284037113189697 seconds
(level _getvalue_z3_call) took 0.2828352451324463 seconds
(level _is_sat_z3_call) took 0.2059028148651123 seconds (unsat)
(level _is_sat_z3_call) took 0.18726372718811035 seconds (unsat)
(level _is_sat_z3_call) took 0.6241359710693359 seconds (sat)
(level _getvalue_z3_call) took 0.18692326545715332 seconds
(level _is_sat_z3_call) took 0.2440180778503418 seconds (sat)
(level _is_sat_z3_call) took 0.22470712661743164 seconds (unsat)
(level _getvalue_z3_call) took 0.24428105354309082 seconds
(level _is_sat_z3_call) took 0.17974472045898438 seconds (unsat)
(level _is_sat_z3_call) took 0.4904510974884033 seconds (sat)
(level _getvalue_z3_call) took 0.4336369037628174 seconds
(level _is_sat_z3_call) took 0.05113697052001953 seconds (unsat)
(level _is_sat_z3_call) took 0.3093106746673584 seconds (sat)
(level _is_

(level _is_sat_z3_call) took 0.6046531200408936 seconds (sat)
(level _getvalue_z3_call) took 0.0008997917175292969 seconds
(level _is_sat_z3_call) took 0.01840496063232422 seconds (unsat)
(level _is_sat_z3_call) took 0.26380157470703125 seconds (sat)
(level _getvalue_z3_call) took 0.005175113677978516 seconds
(level _is_sat_z3_call) took 0.2346503734588623 seconds (unsat)
(level _is_sat_z3_call) took 0.1378793716430664 seconds (sat)
(level _getvalue_z3_call) took 0.020481586456298828 seconds
(level _is_sat_z3_call) took 0.3315401077270508 seconds (unsat)
(level _is_sat_z3_call) took 0.30446648597717285 seconds (sat)
(level _getvalue_z3_call) took 0.17175579071044922 seconds
(level _is_sat_z3_call) took 0.3937497138977051 seconds (unsat)
(level _is_sat_z3_call) took 0.4761965274810791 seconds (sat)
(level _getvalue_z3_call) took 0.13295531272888184 seconds
(level _is_sat_z3_call) took 0.18770909309387207 seconds (unsat)
(level _is_sat_z3_call) took 0.5808389186859131 seconds (sat)
(leve

(level _is_sat_z3_call) took 0.5706958770751953 seconds (unsat)
(level _is_sat_z3_call) took 1.5942070484161377 seconds (sat)
(level _getvalue_z3_call) took 0.2472090721130371 seconds
(level _is_sat_z3_call) took 0.18066835403442383 seconds (sat)
(level _getvalue_z3_call) took 0.04817771911621094 seconds
(level _is_sat_z3_call) took 0.5756850242614746 seconds (unsat)
(level _is_sat_z3_call) took 0.3175044059753418 seconds (sat)
(level _getvalue_z3_call) took 0.01609969139099121 seconds
(level _is_sat_z3_call) took 0.5170269012451172 seconds (unsat)
(level _is_sat_z3_call) took 0.5796711444854736 seconds (sat)
(level _getvalue_z3_call) took 0.32086777687072754 seconds
(level _is_sat_z3_call) took 0.384075403213501 seconds (sat)
(level _getvalue_z3_call) took 0.024982690811157227 seconds
(level _is_sat_z3_call) took 0.1587693691253662 seconds (sat)
(level _getvalue_z3_call) took 0.005292177200317383 seconds
(level _is_sat_z3_call) took 0.34601902961730957 seconds (unsat)
(level _is_sat_z

(level _is_sat_z3_call) took 0.15415716171264648 seconds (sat)
(level _getvalue_z3_call) took 0.5769460201263428 seconds
(level _is_sat_z3_call) took 0.04693865776062012 seconds (unsat)
(level _is_sat_z3_call) took 0.15085387229919434 seconds (sat)
(level _getvalue_z3_call) took 0.09947609901428223 seconds
(level _is_sat_z3_call) took 0.08046150207519531 seconds (unsat)
(level _is_sat_z3_call) took 0.24911832809448242 seconds (sat)
(level _getvalue_z3_call) took 0.04445457458496094 seconds
(level _is_sat_z3_call) took 0.09450364112854004 seconds (unsat)
(level _is_sat_z3_call) took 0.3840487003326416 seconds (sat)
(level _getvalue_z3_call) took 0.005232334136962891 seconds
(level _is_sat_z3_call) took 0.3205296993255615 seconds (unsat)
(level _is_sat_z3_call) took 0.28695249557495117 seconds (sat)
(level _getvalue_z3_call) took 1.0951910018920898 seconds
(level _is_sat_z3_call) took 0.19165372848510742 seconds (unsat)
(level _is_sat_z3_call) took 0.30742430686950684 seconds (sat)
(leve

(level _is_sat_z3_call) took 0.1935131549835205 seconds (sat)
(level _getvalue_z3_call) took 0.08681058883666992 seconds
(level _is_sat_z3_call) took 0.18124771118164062 seconds (sat)
(level _is_sat_z3_call) took 0.15749359130859375 seconds (unsat)
(level _getvalue_z3_call) took 0.23496627807617188 seconds
(level _is_sat_z3_call) took 0.19704341888427734 seconds (unsat)
(level _is_sat_z3_call) took 0.24425292015075684 seconds (sat)
(level _getvalue_z3_call) took 0.14077138900756836 seconds
(level _is_sat_z3_call) took 0.03565669059753418 seconds (unsat)
(level _is_sat_z3_call) took 0.0927276611328125 seconds (sat)
(level _getvalue_z3_call) took 0.16049766540527344 seconds
(level _is_sat_z3_call) took 0.17137408256530762 seconds (unsat)
(level _is_sat_z3_call) took 0.22491121292114258 seconds (sat)
(level _getvalue_z3_call) took 0.03751420974731445 seconds
(level _is_sat_z3_call) took 0.056708574295043945 seconds (unsat)
(level _is_sat_z3_call) took 0.10113239288330078 seconds (sat)
(le

(level _is_sat_z3_call) took 0.24898433685302734 seconds (sat)
(level _getvalue_z3_call) took 0.0742635726928711 seconds
(level _is_sat_z3_call) took 0.2981748580932617 seconds (sat)
(level _getvalue_z3_call) took 0.09619450569152832 seconds
(level _is_sat_z3_call) took 1.0557069778442383 seconds (unsat)
(level _is_sat_z3_call) took 0.209395170211792 seconds (sat)
(level _getvalue_z3_call) took 0.054822444915771484 seconds
(level _is_sat_z3_call) took 0.003965854644775391 seconds (unsat)
(level _is_sat_z3_call) took 0.5139827728271484 seconds (sat)
(level _getvalue_z3_call) took 0.05408143997192383 seconds
(level _is_sat_z3_call) took 0.17931532859802246 seconds (unsat)
(level _is_sat_z3_call) took 0.2963290214538574 seconds (sat)
(level _getvalue_z3_call) took 0.13993144035339355 seconds
(level _is_sat_z3_call) took 0.010413646697998047 seconds (unsat)
(level _is_sat_z3_call) took 0.0986487865447998 seconds (sat)
(level _is_sat_z3_call) took 0.11998128890991211 seconds (sat)
(level _g

(level _is_sat_z3_call) took 0.11549997329711914 seconds (sat)
(level _getvalue_z3_call) took 0.0009698867797851562 seconds
(level _is_sat_z3_call) took 0.02976250648498535 seconds (unsat)
(level _is_sat_z3_call) took 0.14507150650024414 seconds (sat)
(level _is_sat_z3_call) took 0.08978080749511719 seconds (sat)
(level _is_sat_z3_call) took 0.16918468475341797 seconds (sat)
(level _is_sat_z3_call) took 0.3803398609161377 seconds (sat)
(level _is_sat_z3_call) took 0.16057515144348145 seconds (sat)
(level _is_sat_z3_call) took 0.1473093032836914 seconds (sat)
(level _is_sat_z3_call) took 1.2475988864898682 seconds (sat)
(level _is_sat_z3_call) took 0.235213041305542 seconds (sat)
(level _is_sat_z3_call) took 0.5268964767456055 seconds (sat)
(level _is_sat_z3_call) took 0.2888329029083252 seconds (sat)
(level _is_sat_z3_call) took 0.23603463172912598 seconds (sat)
(level _is_sat_z3_call) took 1.1384754180908203 seconds (sat)
(level _is_sat_z3_call) took 0.19604706764221191 seconds (sat)


(level _is_sat_z3_call) took 0.11118912696838379 seconds (sat)
(level _is_sat_z3_call) took 0.26500844955444336 seconds (sat)
(level _is_sat_z3_call) took 0.4597325325012207 seconds (sat)
(level _getvalue_z3_call) took 0.19192028045654297 seconds
(level _is_sat_z3_call) took 0.14362168312072754 seconds (unsat)
(level _is_sat_z3_call) took 0.4382743835449219 seconds (sat)
(level _getvalue_z3_call) took 0.22461557388305664 seconds
(level _is_sat_z3_call) took 0.24375581741333008 seconds (unsat)
(level _is_sat_z3_call) took 0.0931694507598877 seconds (sat)
(level _getvalue_z3_call) took 0.382976770401001 seconds
(level _is_sat_z3_call) took 0.052667856216430664 seconds (unsat)
(level _is_sat_z3_call) took 0.20218539237976074 seconds (sat)
(level _getvalue_z3_call) took 0.27263879776000977 seconds
(level _is_sat_z3_call) took 0.35613274574279785 seconds (unsat)
(level _is_sat_z3_call) took 1.248922348022461 seconds (sat)(level _is_sat_z3_call) took 1.355508804321289 seconds (sat)

(level _

In [16]:
ini_state = (0,1,1) #_SetTargetTemperature_SetMode
found = []
with io.capture_output() as captured:    
    for fin_state in states:
        count = m.generateTestCases(keys=(precondiciones+precondiciones),
                                    targets=(ini_state+fin_state),
                                    testcaseName=f"TRANSITION_{repr_transition(ini_state,fin_state,'SetTargetTemperature')}")
        if count > 0:
            found.append(f"found {count} testcases for transition {repr_transition(ini_state,fin_state,'SetTargetTemperature')}")
        else:
            found.append(f"found no testcases for transition {repr_transition(ini_state,fin_state,'SetTargetTemperature')}")

In [17]:
for line in found:
    print(line)

found no testcases for transition _SetTargetTemperature_SetMode->SetTargetTemperature->vacio
found no testcases for transition _SetTargetTemperature_SetMode->SetTargetTemperature->_SetMode
found no testcases for transition _SetTargetTemperature_SetMode->SetTargetTemperature->_SetTargetTemperature
found True testcases for transition _SetTargetTemperature_SetMode->SetTargetTemperature->_SetTargetTemperature_SetMode
found no testcases for transition _SetTargetTemperature_SetMode->SetTargetTemperature->_StartThermostat
found no testcases for transition _SetTargetTemperature_SetMode->SetTargetTemperature->_StartThermostat_SetMode
found no testcases for transition _SetTargetTemperature_SetMode->SetTargetTemperature->_StartThermostat_SetTargetTemperature
found no testcases for transition _SetTargetTemperature_SetMode->SetTargetTemperature->_StartThermostat_SetTargetTemperature_SetMode


In [18]:
m.goto_snapshot()
m.take_snapshot()
m.callContractFunction("SetMode")
call_pre()

(level _is_sat_z3_call) took 0.05661582946777344 seconds (sat)
(level _getvalue_z3_call) took 0.018872499465942383 seconds
(level _is_sat_z3_call) took 0.04257369041442871 seconds (sat)
(level _getvalue_z3_call) took 0.0009980201721191406 seconds
(level _is_sat_z3_call) took 0.03928375244140625 seconds (sat)
(level _is_sat_z3_call) took 0.0489199161529541 seconds (sat)
(level _getvalue_z3_call) took 0.0028514862060546875 seconds
(level _getvalue_z3_call) took 0.003916501998901367 seconds
(level _is_sat_z3_call) took 0.05973553657531738 seconds (sat)
(level _getvalue_z3_call) took 0.001116037368774414 seconds
(level _is_sat_z3_call) took 0.004079103469848633 seconds (unsat)
(level _is_sat_z3_call) took 0.00698399543762207 seconds (unsat)
(level _is_sat_z3_call) took 0.06808853149414062 seconds (sat)
(level _is_sat_z3_call) took 0.09612011909484863 seconds (sat)
(level _is_sat_z3_call) took 0.09639501571655273 seconds (sat)
(level _is_sat_z3_call) took 0.10827851295471191 seconds (sat)
(

(level _getvalue_z3_call) took 0.007943391799926758 seconds
(level _is_sat_z3_call) took 0.08162665367126465 seconds (unsat)
(level _is_sat_z3_call) took 0.3865816593170166 seconds (sat)
(level _getvalue_z3_call) took 1.7585029602050781 seconds
(level _is_sat_z3_call) took 0.28959012031555176 seconds (unsat)
(level _is_sat_z3_call) took 0.04069638252258301 seconds (sat)
(level _getvalue_z3_call) took 0.005135536193847656 seconds
(level _is_sat_z3_call) took 0.024378538131713867 seconds (unsat)
(level _is_sat_z3_call) took 0.9024553298950195 seconds (sat)
(level _getvalue_z3_call) took 0.01054239273071289 seconds
(level _is_sat_z3_call) took 0.06112813949584961 seconds (unsat)
(level _is_sat_z3_call) took 0.329226016998291 seconds (sat)
(level _getvalue_z3_call) took 0.015868186950683594 seconds
(level _is_sat_z3_call) took 0.061289310455322266 seconds (unsat)
(level _is_sat_z3_call) took 0.1089334487915039 seconds (sat)
(level _getvalue_z3_call) took 0.010747671127319336 seconds
(level

(level _getvalue_z3_call) took 0.1380016803741455 seconds
(level _getvalue_z3_call) took 0.5364480018615723 seconds
(level _is_sat_z3_call) took 0.39685606956481934 seconds (unsat)
(level _is_sat_z3_call) took 0.8265790939331055 seconds (unsat)
(level _is_sat_z3_call) took 0.7743847370147705 seconds (sat)
(level _getvalue_z3_call) took 1.4309642314910889 seconds
(level _is_sat_z3_call) took 0.0154266357421875 seconds (unsat)
(level _is_sat_z3_call) took 0.5142040252685547 seconds (sat)
(level _getvalue_z3_call) took 0.5025513172149658 seconds
(level _is_sat_z3_call) took 0.4951176643371582 seconds (unsat)
(level _is_sat_z3_call) took 0.9370472431182861 seconds (sat)
(level _getvalue_z3_call) took 0.025707006454467773 seconds
(level _is_sat_z3_call) took 0.005243778228759766 seconds (unsat)
(level _is_sat_z3_call) took 0.18777084350585938 seconds (sat)
(level _getvalue_z3_call) took 0.030618667602539062 seconds
(level _is_sat_z3_call) took 0.7231407165527344 seconds (unsat)
(level _is_s

(level _is_sat_z3_call) took 0.269930362701416 seconds (sat)
(level _getvalue_z3_call) took 0.015289783477783203 seconds
(level _is_sat_z3_call) took 0.01527261734008789 seconds (unsat)
(level _is_sat_z3_call) took 0.05595040321350098 seconds (sat)
(level _getvalue_z3_call) took 0.2655453681945801 seconds
(level _is_sat_z3_call) took 0.020387649536132812 seconds (unsat)
(level _is_sat_z3_call) took 0.09733986854553223 seconds (sat)
(level _getvalue_z3_call) took 0.020427227020263672 seconds
(level _is_sat_z3_call) took 0.020364046096801758 seconds (unsat)
(level _is_sat_z3_call) took 0.08143043518066406 seconds (sat)
(level _getvalue_z3_call) took 2.2500452995300293 seconds
(level _is_sat_z3_call) took 0.40214061737060547 seconds (unsat)
(level _is_sat_z3_call) took 0.7458763122558594 seconds (sat)
(level _getvalue_z3_call) took 0.6383802890777588 seconds
(level _is_sat_z3_call) took 0.5124688148498535 seconds (unsat)
(level _is_sat_z3_call) took 0.4317901134490967 seconds (sat)
(level

(level _is_sat_z3_call) took 0.30906152725219727 seconds (sat)
(level _getvalue_z3_call) took 0.01597118377685547 seconds
(level _is_sat_z3_call) took 0.2388474941253662 seconds (sat)
(level _getvalue_z3_call) took 0.6034338474273682 seconds
(level _is_sat_z3_call) took 0.08640718460083008 seconds (unsat)
(level _is_sat_z3_call) took 0.12641334533691406 seconds (sat)
(level _getvalue_z3_call) took 0.7248499393463135 seconds
(level _is_sat_z3_call) took 0.21297621726989746 seconds (sat)
(level _getvalue_z3_call) took 0.04643440246582031 seconds
(level _is_sat_z3_call) took 0.2527735233306885 seconds (unsat)
(level _is_sat_z3_call) took 0.8690910339355469 seconds (sat)
(level _getvalue_z3_call) took 0.12189459800720215 seconds
(level _is_sat_z3_call) took 0.015331745147705078 seconds (unsat)
(level _is_sat_z3_call) took 0.7666075229644775 seconds (sat)
(level _getvalue_z3_call) took 0.06168508529663086 seconds
(level _is_sat_z3_call) took 1.1084110736846924 seconds (unsat)
(level _is_sat

(level _is_sat_z3_call) took 0.0971670150756836 seconds (unsat)
(level _is_sat_z3_call) took 0.07228779792785645 seconds (sat)
(level _getvalue_z3_call) took 0.0658562183380127 seconds
(level _is_sat_z3_call) took 0.038520097732543945 seconds (unsat)
(level _is_sat_z3_call) took 0.03944540023803711 seconds (sat)
(level _getvalue_z3_call) took 0.05621528625488281 seconds
(level _is_sat_z3_call) took 0.41794323921203613 seconds (unsat)
(level _is_sat_z3_call) took 0.07781553268432617 seconds (sat)
(level _getvalue_z3_call) took 0.13193845748901367 seconds
(level _is_sat_z3_call) took 0.31538915634155273 seconds (unsat)
(level _is_sat_z3_call) took 0.508155107498169 seconds (sat)
(level _getvalue_z3_call) took 0.19193792343139648 seconds
(level _is_sat_z3_call) took 0.04588770866394043 seconds (unsat)
(level _is_sat_z3_call) took 0.14867186546325684 seconds (sat)
(level _getvalue_z3_call) took 0.1122283935546875 seconds
(level _is_sat_z3_call) took 0.3118774890899658 seconds (unsat)
(leve

(level _is_sat_z3_call) took 0.5435326099395752 seconds (unsat)
(level _is_sat_z3_call) took 0.5248041152954102 seconds (sat)
(level _getvalue_z3_call) took 0.29101061820983887 seconds
(level _is_sat_z3_call) took 0.31063055992126465 seconds (unsat)
(level _is_sat_z3_call) took 0.13872027397155762 seconds (sat)(level _is_sat_z3_call) took 0.08749532699584961 seconds (sat)
(level _getvalue_z3_call) took 0.025568723678588867 seconds

(level _is_sat_z3_call) took 0.08226370811462402 seconds (sat)
(level _getvalue_z3_call) took 0.010570049285888672 seconds
(level _is_sat_z3_call) took 0.09639978408813477 seconds (unsat)
(level _getvalue_z3_call) took 0.10372281074523926 seconds
(level _is_sat_z3_call) took 0.44499874114990234 seconds (sat)
(level _getvalue_z3_call) took 0.010343551635742188 seconds
(level _is_sat_z3_call) took 0.27353572845458984 seconds (unsat)
(level _is_sat_z3_call) took 0.661818265914917 seconds (unsat)
(level _is_sat_z3_call) took 0.032265663146972656 seconds (sat)
(l

(level _is_sat_z3_call) took 0.5385472774505615 seconds (sat)
(level _getvalue_z3_call) took 0.004993915557861328 seconds
(level _is_sat_z3_call) took 0.010218143463134766 seconds (unsat)
(level _is_sat_z3_call) took 0.10486149787902832 seconds (sat)
(level _getvalue_z3_call) took 0.04989480972290039 seconds
(level _is_sat_z3_call) took 0.035660505294799805 seconds (sat)
(level _getvalue_z3_call) took 0.005277395248413086 seconds
(level _is_sat_z3_call) took 0.030625581741333008 seconds (unsat)
(level _is_sat_z3_call) took 0.4635615348815918 seconds (sat)
(level _getvalue_z3_call) took 0.009124040603637695 seconds
(level _is_sat_z3_call) took 0.05072379112243652 seconds (unsat)
(level _is_sat_z3_call) took 0.118988037109375 seconds (sat)
(level _getvalue_z3_call) took 0.008185386657714844 seconds
(level _is_sat_z3_call) took 0.006377696990966797 seconds (unsat)
(level _is_sat_z3_call) took 0.12756681442260742 seconds (sat)
(level _getvalue_z3_call) took 0.0011658668518066406 seconds
(l

(level _is_sat_z3_call) took 0.07674956321716309 seconds (sat)
(level _getvalue_z3_call) took 0.015824317932128906 seconds
(level _is_sat_z3_call) took 0.2609446048736572 seconds (unsat)
(level _is_sat_z3_call) took 0.10183143615722656 seconds (sat)
(level _getvalue_z3_call) took 0.010321378707885742 seconds
(level _is_sat_z3_call) took 0.39571690559387207 seconds (unsat)
(level _is_sat_z3_call) took 0.06893229484558105 seconds (sat)
(level _getvalue_z3_call) took 0.001316070556640625 seconds
(level _is_sat_z3_call) took 0.006616115570068359 seconds (unsat)
(level _is_sat_z3_call) took 0.10694622993469238 seconds (sat)
(level _is_sat_z3_call) took 0.1384108066558838 seconds (sat)
(level _is_sat_z3_call) took 0.09570050239562988 seconds (sat)
(level _is_sat_z3_call) took 0.0905466079711914 seconds (sat)
(level _is_sat_z3_call) took 0.07803821563720703 seconds (sat)
(level _is_sat_z3_call) took 0.09978842735290527 seconds (sat)
(level _is_sat_z3_call) took 0.10411453247070312 seconds (sa

In [19]:
ini_state = (0,1,1) #_SetTargetTemperature_SetMode
found = []

for fin_state in states:
    with io.capture_output() as captured:    
        count = m.generateTestCases(keys=(precondiciones+precondiciones),
                                    targets=(ini_state+fin_state),
                                    testcaseName=f"TRANSITION_{repr_transition(ini_state,fin_state,'SetMode')}")
        if count > 0:
            found.append(f"found {count} testcases for transition {repr_transition(ini_state,fin_state,'SetMode')}")
        else:
            found.append(f"found no testcases for transition {repr_transition(ini_state,fin_state,'SetMode')}")

In [20]:
for line in found:
    print(line)

found no testcases for transition _SetTargetTemperature_SetMode->SetMode->vacio
found no testcases for transition _SetTargetTemperature_SetMode->SetMode->_SetMode
found no testcases for transition _SetTargetTemperature_SetMode->SetMode->_SetTargetTemperature
found True testcases for transition _SetTargetTemperature_SetMode->SetMode->_SetTargetTemperature_SetMode
found no testcases for transition _SetTargetTemperature_SetMode->SetMode->_StartThermostat
found no testcases for transition _SetTargetTemperature_SetMode->SetMode->_StartThermostat_SetMode
found no testcases for transition _SetTargetTemperature_SetMode->SetMode->_StartThermostat_SetTargetTemperature
found no testcases for transition _SetTargetTemperature_SetMode->SetMode->_StartThermostat_SetTargetTemperature_SetMode
